In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import brute, fmin

In [4]:
def Heston_char_function(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """Valuation of European call option in Hetson model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value."""
    c1 = kappa_v * theta_v
    c2 = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
    )
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) / (
        kappa_v - rho * sigma_v * u * 1j - c2
    )
    H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
    )
    H2 = (
        (kappa_v - rho * sigma_v * u * 1j + c2)
        / sigma_v**2
        * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T)))
    )
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

In [5]:
def Heston_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """
    Fourier-based approach for Lewis (2001): Integration function.
    """
    char_func_value = Heston_char_function(
        u - 1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0
    )
    int_func_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    )
    return int_func_value

In [6]:
def Heston_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):

    int_value = quad(
        lambda u: Heston_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value

def Heston_put_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):

    int_value = quad(
        lambda u: Heston_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    put_value = call_value - S0 + np.exp(-r * T)
    return put_value

In [7]:
S0 = 232.90  # The SM stock is currently trading at this price
r = 0.015
number_of_annual_trading_days = 250
#Price params
call_market_price = np.array([10.52, 10.05, 7.75, 6.01, 4.75, 16.78, 17.65, 16.86, 16.05, 15.1,27.92, 24.12, 22.97, 21.75, 18.06])
put_market_price = np.array([4.32, 5.2, 6.45, 7.56,8.78, 11.03,12.15,13.37,14.75,15.62,14.53,16.25,17.22,18.74,19.73])

#Maturity params
call_market_maturity = [*np.array([15]*5) , *np.array([60]*5) , *np.array([120]*5)]
call_market_maturity = [x / number_of_annual_trading_days for x in call_market_maturity]

put_market_maturity= [*np.array([15]*5) , *np.array([60]*5) , *np.array([120]*5)]
put_market_maturity = [x / number_of_annual_trading_days for x in put_market_maturity]

#Strike params
call_market_strike = np.array([227.5,230,232.5,235,237.5,227.5,230,232.5,235,237.5,227.5,230,232.5,235,237.5])
put_market_strike = call_market_strike

#Interest rate
call_interest = np.array([r]*15)
put_interest = call_interest

#Form table of call and put option
#call_option = pd.DataFrame({'Strike': call_market_strike, 'T': call_market_maturity, 'r': call_interest, 'Call': call_market_price})
#put_option = pd.DataFrame({'Strike': put_market_strike, 'T': put_market_maturity, 'r': put_interest, 'Put': put_market_price})


In [8]:
i = 0
min_MSE = 500

def Heston_call_error_function(p0):
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v**2:
        return 500.0
    se = []
    j = 0
    for st in call_market_strike:
        model_value = Heston_call_value(
            S0,
            st,
            call_market_maturity[j],
            call_interest[j],
            kappa_v,
            theta_v,
            sigma_v,
            rho,
            v0,
        )
        se.append((model_value - call_market_price[j]) ** 2)
        j = j + 1
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print("%4d |" % i, np.array(p0), "| %7.3f | %7.3f" % (MSE, min_MSE))
    i += 1
    return MSE

Running against something


In [9]:
def Heston_call_calibration_full():
    """Calibrates Heston (1993) stochastic volatility model to market quotes."""
    # First run with brute force
    # (scan sensible regions, for faster convergence)
    print("First step: Brute force")
    p0 = brute(
        Heston_call_error_function,
        (
            #(22.5, 28.6, 2.5),  # kappa_v
            #(0.11, 0.14, 0.01),  # theta_v
            #(0.05, 0.25, 0.1),  # sigma_v
            #(-0.15, 0.30, 0.15),  # rho
            #(0.021, 0.031, 0.001),  # v0

            (15, 18.6, 2.5),  # kappa_v
            (0.12, 0.13, 0.01),  # theta_v
            (0.8, 0.9, 0.1),  # sigma_v
            (-0.95, -0.50, 0.15),  # rho
            (0.098, 0.1, 0.001), #v0
        ),
        finish=None,
    )
    print("Second step:")
    # Second run with local, convex minimization
    # (we dig deeper where promising results)
    opt = fmin(
        Heston_call_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=750, maxfun=900
    )
    return opt

opt = Heston_call_calibration_full()

First step: Brute force
   0 | [15.     0.12   0.8   -0.95   0.098] |   1.633 |   1.633
  25 | [17.5    0.12   0.8   -0.65   0.099] |   1.545 |   1.500
Second step:
  50 | [15.011418    0.12888746  0.81222496 -0.98351714  0.09750541] |   1.484 |   1.482
  75 | [14.15102175  0.12627093  0.82998247 -0.96057762  0.10564292] |   1.471 |   1.471
 100 | [14.1143109   0.1267027   0.82326926 -0.97755521  0.10536363] |   1.471 |   1.471
 125 | [13.52119403  0.12666687  0.81485909 -0.99685208  0.10659536] |   1.470 |   1.470
 150 | [13.48606724  0.12667668  0.81314383 -0.99967685  0.10658045] |   1.470 |   1.470
 175 | [13.4830481   0.12664077  0.8125185  -0.99996829  0.10661212] |   1.470 |   1.470
 200 | [13.483903    0.1266439   0.81260057 -0.99999782  0.1066309 ] |   1.470 |   1.470
 225 | [13.4957241   0.12665066  0.8128261  -0.99999757  0.10660439] |   1.470 |   1.470
 250 | [13.63784716  0.12656865  0.81351601 -0.99998747  0.10650653] |   1.470 |   1.470
 275 | [15.44987406  0.12531444  0

/tmp/ipykernel_6695/1804100815.py:26: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  opt = fmin(
